# Training ```FFANN```

Let's augment the ```FFANN``` class in order to be able to train it.

This is still a work in progress, but at least it can "fit" the *XOR*-gate and a linear function.

In [1]:
import FeedForwardANN as FFANN
import numpy as np

In [2]:
lin=FFANN.linearActivation()
sig=FFANN.sigmoidActivation()



In [3]:
# data_in=[[_] for _ in np.linspace(0,1,250)]
# data_out=[[2*_] for _ in np.linspace(0,1,250)]
# brain=FFANN.FFANN(1,1,[3,2,1],[lin,lin,lin,lin,lin])
# brain.init_params(-1,1)


data_in=[[1,1],[0,0],[1,0],[0,1]]
data_out=[[0],[0],[1],[1]]
brain=FFANN.FFANN(2,1,[3],[sig,lin])
brain.init_params(-1e-1,1e-1)

In [4]:
def Q_i(signal,target):
    return (signal-target)**2.

def dQds_i(signal,target):
    return 2.*(signal-target)

In [5]:
Q=FFANN.loss(Q_i, dQds_i, brain)
# strategy=FFANN.VanillaSGD(brain,Q,alpha=1e-1)
# strategy=FFANN.RMSpropSGD(brain,Q,gamma=0.995,epsilon=1e-4,alpha=1e-2)
# strategy=FFANN.AdaDeltaSGD(brain,Q,gamma=0.995,epsilon=1e-4,alpha=1)
# strategy=FFANN.AdamSGD(brain,Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)
strategy=FFANN.AdaMaxSGD(brain,Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)


In [6]:
brain.SGD(strategy, data_in, data_out, abs_tol=1e-3, rel_tol=1e-3, step_break=250,max_step=50000)

12935

In [7]:
meanQ=0
maxQ=0
for i,_ in enumerate(data_in):
    Qi=Q(brain(_),data_out[i])
    print('point:',_,'\n'
       'target:',data_out[i],
         'FFANN gives:',brain(_),
         'loss:',Qi)
    print('')
    if Qi > maxQ:
        maxQ=Qi
    meanQ+=Q(brain(_),data_out[i])
    
print('mean loss:', meanQ/float(len(data_out)))
print('max loss:', maxQ)


point: [1, 1] 
target: [0] FFANN gives: [0.00015131410538207768] loss: 2.289595848757851e-08

point: [0, 0] 
target: [0] FFANN gives: [6.364296599634112e-05] loss: 4.050427120811432e-09

point: [1, 0] 
target: [1] FFANN gives: [0.9999891583412591] loss: 1.1754156425367444e-10

point: [0, 1] 
target: [1] FFANN gives: [0.9999508102796636] loss: 2.4196285867739e-09

mean loss: 7.37088893985438e-09
max loss: 2.289595848757851e-08
